In [1]:
import os
import numpy as np 
import pandas as pd

from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

from sklearn.multioutput import MultiOutputRegressor
import matplotlib.pyplot as plt

In [11]:
X_train = pd.read_csv("data/ancestry_train.data", sep = " ", header=None).values
y_train = pd.read_csv("data/ancestry_train.solution", sep = " ", header=None).values

X_test = pd.read_csv("data/ancestry_test.data", sep = " ", header=None).values

In [3]:
X_train

array([[0, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [12]:
np.round(y_train, 3)

array([[0.198, 0.75 , 0.052],
       [0.858, 0.142, 0.   ],
       [0.761, 0.237, 0.001],
       ...,
       [0.001, 0.183, 0.817],
       [0.016, 0.591, 0.393],
       [0.   , 0.007, 0.993]])

In [4]:
X_test

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 2, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 2, 1, 1],
       [0, 0, 2, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

# cost function

In [10]:
def cost(y_true, y_hat):
    mse = -np.log10(np.mean((y_true-y_hat)**2)+1e-5)
    return(mse)

In [ ]:
###### MODEL 1: SIMPLE RANDOM FOREST 

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import accuracy_score

In [ ]:
# Create the Random Forest classifier
clf = RandomForestRegressor(n_estimators=50, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_train)

print(y_pred)
cost(y_train, y_pred)

In [ ]:
y_test = clf.predict(X_test)
pd.DataFrame(y_test).to_csv(f"predictions.csv", sep = " ", header = None, index = None)
os.system("zip -r predictions.zip predictions.csv")

In [10]:
#another attempt: GLM

from sklearn.mixture import GaussianMixture
import numpy as np

# Assuming you have SNP data (X_snp) and ancestry labels (y_labels)

# Initialize GMM
num_ancestries = 3  # Update this based on the number of ancestries you want to identify
gmm = GaussianMixture(n_components=num_ancestries, random_state=42)

# Train the model
gmm.fit(X_train)

# Assign labels to individuals based on highest probability cluster
predicted_labels = gmm.predict(X_train)

# For new individuals without labels:
# Assuming new_data contains SNP data of new individuals
predicted_new_labels = gmm.predict(X_test)

# Calculate ancestry proportions for each individual
probs = gmm.predict_proba(X_train)
# probs contains the probability of each individual belonging to each ancestry

print(probs)

# Assess performance (if true labels are available)
#accuracy = np.mean(predicted_labels == y_labels)

[[3.68791317e-51 0.00000000e+00 1.00000000e+00]
 [4.78632874e-74 0.00000000e+00 1.00000000e+00]
 [1.52856970e-66 0.00000000e+00 1.00000000e+00]
 ...
 [1.06710770e-78 1.00000000e+00 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.69328467e-55 1.00000000e+00 0.00000000e+00]]


In [53]:
#another attempt: independent logistic regression. this scores well (2.4). best so far. 
from sklearn.linear_model import LinearRegression

# Initialize three linear regression models for each ancestral group
linreg_ancestry_1 = LinearRegression()
linreg_ancestry_2 = LinearRegression()
linreg_ancestry_3 = LinearRegression()

# Train each model separately on the corresponding column of y_labels
linreg_ancestry_1.fit(X_train, y_train[:, 0])  # First column represents ancestry 1
linreg_ancestry_2.fit(X_train, y_train[:, 1])  # Second column represents ancestry 2
linreg_ancestry_3.fit(X_train, y_train[:, 2])  # Third column represents ancestry 3

# Predict values for each ancestral group for the test set
y_pred_ancestry_1 = linreg_ancestry_1.predict(X_test)
y_pred_ancestry_2 = linreg_ancestry_2.predict(X_test)
y_pred_ancestry_3 = linreg_ancestry_3.predict(X_test)


In [4]:
all_predictions = np.column_stack((y_pred_ancestry_1, y_pred_ancestry_2, y_pred_ancestry_3))

# Normalize along each row to ensure each row sums to 1

all_predictions[all_predictions < 0] = 0

normalized_predictions = all_predictions / np.sum(all_predictions, axis=1, keepdims=True)

print(normalized_predictions)

pd.DataFrame(normalized_predictions).to_csv(f"predictions.csv", sep = " ", header = None, index = None)
os.system("zip -r predictions.zip predictions.csv")

#plt.scatter (X_test, y_pred_ancestry_1)

NameError: name 'y_pred_ancestry_1' is not defined

In [6]:
from sklearn.linear_model import LinearRegression

#this is the same relatively as above, performs a bit worse. 

# Assuming X_train contains SNP data and y_train contains proportions of ancestries A, B, C

# Initialize multivariate multiple regression model
multivariate_regression = LinearRegression()

# Fit the model with SNP data to predict proportions of ancestries A, B, C
multivariate_regression.fit(X_train, y_train)  # y_train should contain proportions of A, B, C as columns

# Predict proportions for test SNP data
predictions = multivariate_regression.predict(X_test)

predictions[predictions < 0] = 0


normalized_predictions = predictions / np.sum(predictions, axis=1, keepdims=True)

print(normalized_predictions)

pd.DataFrame(normalized_predictions).to_csv(f"predictions.csv", sep = " ", header = None, index = None)
os.system("zip -r predictions.zip predictions.csv")

print(predictions)

[[0.         0.50018079 0.49981921]
 [0.9926543  0.0073457  0.        ]
 [0.         0.         1.        ]
 ...
 [0.02971191 0.47441637 0.49587171]
 [0.02313964 0.         0.97686036]
 [0.04434458 0.10199848 0.85365694]]
updating: predictions.csv (deflated 54%)
[[0.         0.540707   0.54031613]
 [1.01392287 0.00750309 0.        ]
 [0.         0.         1.09374057]
 ...
 [0.0297116  0.47441147 0.49586658]
 [0.02375787 0.         1.00295937]
 [0.04434195 0.10199243 0.85360628]]


In [9]:
#this gettst a score of 2.8 
from sklearn.linear_model import TweedieRegressor
from sklearn.multioutput import MultiOutputRegressor
import numpy as np
import pandas as pd
import os

# Assuming X_train contains SNP data and y_train contains proportions of ancestries A, B, C in three columns

# Initialize Tweedie regression model with appropriate settings
tweedie_regression = TweedieRegressor(power=0, link='identity', alpha=0.5)

# Wrap the model with MultiOutputRegressor for multi-output regression
multi_output_model = MultiOutputRegressor(tweedie_regression)

# Fit the model with SNP data to predict proportions of ancestries A, B, C
multi_output_model.fit(X_train, y_train)



MultiOutputRegressor(estimator=TweedieRegressor(alpha=0.5, link='identity',
                                                power=0))

In [12]:
predictions = multi_output_model.predict(X_test)

# Replace negative predictions with 0
predictions[predictions < 0] = 0

# Normalize predictions for each row (across the three columns)
normalized_predictions = predictions / np.sum(predictions, axis=1, keepdims=True)

pd.DataFrame(normalized_predictions).to_csv(f"predictions.csv", sep = " ", header = None, index = None)
os.system("zip -r predictions.zip predictions.csv")

print(normalized_predictions)

updating: predictions.csv (deflated 54%)
[[0.         0.4816134  0.5183866 ]
 [0.97679839 0.01478855 0.00841306]
 [0.         0.00299924 0.99700076]
 ...
 [0.04435038 0.43797064 0.51767898]
 [0.00212866 0.02028689 0.97758444]
 [0.03895111 0.12682317 0.83422572]]


In [13]:


#attempt 11/16: try to make all labels > 0 before I make predictions 
y_train_clipped = np.clip(y_train, 0.000001, 0.999999)  # Avoids exact 0 or 1
def apply_log(val):
    return np.log(val / (1 - val))

# Apply the function to all columns
y_train = apply_log(y_train_clipped)


[[ -1.39594929   1.09686919  -2.90580617]
 [  1.7981697   -1.80168823  -7.7551872 ]
 [  1.16062398  -1.16808911  -6.60380831]
 ...
 [ -7.19786233  -1.49896121   1.49395977]
 [ -4.10260007   0.36689711  -0.4345928 ]
 [-13.81550956  -4.95233576   4.95220542]]


In [15]:
from sklearn.linear_model import LinearRegression

#this doesn't work at all. 
# Initialize three linear regression models for each ancestral group
linreg_ancestry_1 = LinearRegression()
linreg_ancestry_2 = LinearRegression()
linreg_ancestry_3 = LinearRegression()

# Train each model separately on the corresponding column of y_labels
linreg_ancestry_1.fit(X_train, y_train[:, 0])  # First column represents ancestry 1
linreg_ancestry_2.fit(X_train, y_train[:, 1])  # Second column represents ancestry 2
linreg_ancestry_3.fit(X_train, y_train[:, 2])  # Third column represents ancestry 3

# Predict values for each ancestral group for the test set
y_pred_ancestry_1 = linreg_ancestry_1.predict(X_test)
y_pred_ancestry_2 = linreg_ancestry_2.predict(X_test)
y_pred_ancestry_3 = linreg_ancestry_3.predict(X_test)

all_predictions = np.column_stack((y_pred_ancestry_1, y_pred_ancestry_2, y_pred_ancestry_3))



In [16]:
def return_toscale(y):
    return 1 / (np.exp(-y) + 1)

all_predictions = return_toscale(all_predictions)

print(all_predictions)

[[1.13620897e-03 5.42451250e-01 8.78146134e-01]
 [9.80941114e-01 1.78404116e-02 9.14517930e-06]
 [5.05674566e-06 1.95833512e-02 9.88957387e-01]
 ...
 [2.42721613e-03 4.26680888e-01 7.16401028e-01]
 [2.37133835e-05 1.96596388e-02 9.50705984e-01]
 [9.27706788e-05 4.20877985e-02 8.98895293e-01]]


import matplotlib.pyplot as plt

# Assuming 'reduced_snp_data' contains the transformed data from PCA

# Plotting the first two principal components
plt.figure(figsize=(8, 6))
plt.scatter(reduced_snp_data[:, 0], reduced_snp_data[:, 1], alpha=0.5)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Visualization of Principal Components')
plt.grid()
plt.show()

# baseline

In [12]:
base = np.ones((y_train.shape[0], y_train.shape[1])) * (1/3)
cost(y_train, base)




0.8401067369385945

In [13]:
y_test = np.ones((X_test.shape[0], y_train.shape[1])) * (1/3)

# save and zip the file

In [14]:
pd.DataFrame(y_test).to_csv(f"predictions.csv", sep = " ", header = None, index = None)
os.system("zip -r predictions.zip predictions.csv")

updating: predictions.csv (deflated 100%)


0